In [18]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.callbacks import EarlyStopping
from keras import backend as K
import pickle
import keras
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt

In [19]:
with open("trainX.pickle","rb") as f:
    X = pickle.load(f)
    
with open("trainY.pickle","rb") as f:
    Y = pickle.load(f)

In [20]:
train_x= X[:40000]
validate_x= X[40000:50000]
test_x= X[50000:]

train_y= Y[:40000]
validate_y= Y[40000:50000]
test_y= Y[50000:]

In [21]:
num_classes= 10
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
validate_x = validate_x.astype('float32')

train_x /= 255
test_x /= 255
validate_x/= 255

print('x_train shape:', train_x.shape)
print(train_x.shape[0], 'train samples')
print(test_x.shape[0], 'test samples')
print(validate_x.shape[0], 'validation samples')

# convert class vectors to binary class matrices
train_y = keras.utils.to_categorical(train_y, num_classes)
test_y = keras.utils.to_categorical(test_y, num_classes)
validate_y = keras.utils.to_categorical(validate_y, num_classes)

x_train shape: (40000, 28, 28)
40000 train samples
10000 test samples
10000 validation samples


In [22]:
train_x = train_x.reshape((train_x.shape[0], 28, 28, 1))
test_x = test_x.reshape((test_x.shape[0], 28, 28, 1))
validate_x = np.reshape(validate_x, (10000, 28, 28, 1))

In [23]:
model = Sequential() 
model.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1), activation='relu')) # add input layer

model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Flatten())

model.add(Dense(output_dim=128, kernel_initializer ='uniform', activation='relu'))

model.add(Dense(output_dim=10, kernel_initializer ='uniform', activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_monitor = EarlyStopping(patience=3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


C:\Users\Nishant Sirbhai\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., activation="relu")`
  
C:\Users\Nishant Sirbhai\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=128)`
  import sys
C:\Users\Nishant Sirbhai\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="softmax", units=10)`
  if __name__ == '__main__':


In [24]:
model.fit(train_x,train_y,validation_data=(test_x, test_y), epochs=10, verbose=True, callbacks=[early_stopping_monitor])
scores = model.evaluate(test_x,test_y,verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100)) 

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 42s 1ms/step - loss: 0.4906 - acc: 0.8236 - val_loss: 0.3400 - val_acc: 0.8777
Epoch 2/10
40000/40000 [==============================] - 43s 1ms/step - loss: 0.3139 - acc: 0.8872 - val_loss: 0.3094 - val_acc: 0.8873
Epoch 3/10
40000/40000 [==============================] - 39s 978us/step - loss: 0.2630 - acc: 0.9028 - val_loss: 0.2773 - val_acc: 0.8976
Epoch 4/10
40000/40000 [==============================] - 39s 978us/step - loss: 0.2267 - acc: 0.9164 - val_loss: 0.2591 - val_acc: 0.9078
Epoch 5/10
40000/40000 [==============================] - 40s 988us/step - loss: 0.1950 - acc: 0.9285 - val_loss: 0.2625 - val_acc: 0.9044
Epoch 6/10
40000/40000 [==============================] - 40s 1ms/step - loss: 0.1677 - acc: 0.9388 - val_loss: 0.2528 - val_acc: 0.9115
Epoch 7/10
40000/40000 [==============================] - 40s 1ms/step - loss: 0.1471 - acc: 0.9456 - val_loss: 0.2712 - v

In [25]:
with open('testX.pickle','rb') as f:
    pred=pickle.load(f)
pred=np.reshape(pred,(10000,28,28,1))
Prediction=model.predict_classes(pred)
Prediction[8788]

5

In [28]:
with open('Predictions.pickle', 'wb') as f:
    pickle.dump(Prediction, f)
